# SparkSession

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, regexp_replace, count, row_number, when, rand, floor
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
spark = SparkSession.builder \
    .appName("Dataload") \
    .getOrCreate()

# Bronze

원시 데이터를 그대로 저장하는 단계

In [0]:
catalog = "1dt_team8_databricks"
schema = "`movielens-32m`"
volume_path = f"{catalog}.{schema}"

try:
    ratings = spark.read.table(f"{volume_path}.ratings")
    movies = spark.read.table(f"{volume_path}.movies")
    tags = spark.read.table(f"{volume_path}.tags")
    links = spark.read.table(f"{volume_path}.links")
    print("Data loaded successfully from Unity Catalog Volume.")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
    print(f"Please ensure CSV files (movies.csv, ratings.csv, links.csv, tags.csv) exist in {volume_path}")

# Silver

정제 및 필터링 된 데이터 저장 단계

In [0]:
ratings = ratings.dropna().dropDuplicates()
movies = movies.dropna().dropDuplicates()

ratings = ratings.filter((ratings.rating >= 0) & (ratings.rating <= 5))

In [0]:
movies.show(5)

In [0]:
ratings.show(5)

## 전처리

In [0]:
# Ensure ratings DataFrame is loaded before proceeding
if 'ratings' in locals():
    tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
    tmp2 = ratings.groupBy("movieId").count().toPandas()['count'].min()
    print('For the users that rated movies and the movies that were rated:')
    print('Minimum number of ratings per user is {}'.format(tmp1))
    print('Minimum number of ratings per movie is {}'.format(tmp2))
else:
    print("Ratings DataFrame not loaded. Please check the data loading step.")

### 유저 수

In [0]:
if 'ratings' in locals():
    tmp_q1 = ratings.select('userid').distinct().count()
    print('There totally have {} users'.format(tmp_q1))
else:
    print("Ratings DataFrame not loaded.")

### 생성연도 새로운 컬럼으로

In [0]:
# 연도 추출 → year 컬럼 생성 (raw string 사용)
movies_df = movies.withColumn(
    "year",
    regexp_extract(col("title"), r"\((\d{4})\)\s*$", 1).cast("int")
)

In [0]:
movies_df.show(5)

### 연도별 영화개수

In [0]:
# 1. 연도별 영화 개수 집계
yearly_movie_count_df = movies_df.groupBy("year").agg(count("*").alias("movie_count"))
yearly_movie_count_df = yearly_movie_count_df.orderBy("year")

# 2. Pandas로 변환
yearly_movie_count_pd = yearly_movie_count_df.toPandas()

# 3. Null 연도 제외 (옵션)
yearly_movie_count_pd = yearly_movie_count_pd.dropna(subset=["year"])

# 4. 막대그래프 그리기
plt.figure(figsize=(15, 6))
plt.bar(yearly_movie_count_pd['year'], yearly_movie_count_pd['movie_count'], color='skyblue')

plt.title("Number of Movies per Year")
plt.xlabel("Year")
plt.ylabel("Movie Count")
plt.xticks()  # 연도 회전
plt.tight_layout()
plt.show()

## Join

In [0]:
movies_with_ratings = movies_df.join(ratings, on="movieId", how="left_outer") \
                             .filter(col("rating").isNotNull())

movies_with_ratings.show(5)

In [0]:
# 1. 영화별 평점 개수 집계
rating_counts_df = movies_with_ratings.groupBy("movieId").agg(count("rating").alias("rating_count"))

# 2. 평점 수 k개 이상인 영화만 필터링
k = 20
valid_movies_df = rating_counts_df.filter(col("rating_count") >= k)
non_valid_movies_df = rating_counts_df.filter(col("rating_count") < k)

# 3. 원본 데이터에서 평점 수 5개 초과인 영화만 남기기
filtered_movies_with_ratings = movies_with_ratings.join(valid_movies_df.select("movieId"), on="movieId", how="inner")
non_valid_movies_df = movies_with_ratings.join(non_valid_movies_df.select("movieId"), on="movieId", how="inner")

In [0]:
print("필터링 된 영화 수: {}".format(filtered_movies_with_ratings.select("movieId").distinct().count()))
print("필터링 안 된 영화 수: {}".format(non_valid_movies_df.select("movieId").distinct().count()))

### 연도별 필터링 된 영화 수

In [0]:
# 1. 연도별 영화 개수 집계
yearly_movie_count_filter_df = filtered_movies_with_ratings.groupBy("year").agg(count("*").alias("movie_count"))
yearly_movie_count_filter_df = yearly_movie_count_filter_df.orderBy("year")

# 2. Pandas로 변환
yearly_movie_count_filter_pd = yearly_movie_count_filter_df.toPandas()

# 3. Null 연도 제외 (옵션)
yearly_movie_count_filter_pd = yearly_movie_count_filter_pd.dropna(subset=["year"])

# 4. 막대그래프 그리기
plt.figure(figsize=(15, 6))
plt.bar(yearly_movie_count_filter_pd['year'], yearly_movie_count_filter_pd['movie_count'], color='skyblue')

plt.title("Number of Movies per Year")
plt.xlabel("Year")
plt.ylabel("Movie Count")
plt.xticks()  # 연도 회전
plt.tight_layout()
plt.show()

### 유저별 리뷰 수 20개 미만 제외

In [0]:
# 1. user별 리뷰 수 집계
user_counts = filtered_movies_with_ratings.groupBy("userId").agg(count("*").alias("review_count"))

# 2. 리뷰 수 20개 이상인 userId만 필터링
valid_users = user_counts.filter("review_count >= 20").select("userId")

# 3. 원본 DataFrame에서 valid_users에 해당하는 user만 남기기 (inner join)
filtered_df = filtered_movies_with_ratings.join(valid_users, on="userId", how="inner")

# train/val/test split

유저별로 6:2:2로 나눔

In [0]:
# 1) 유저별로 랜덤 정렬해서 row 번호 생성 (datetime 없으면 rand 사용)
window = Window.partitionBy("userId").orderBy(rand())

filtered_df = filtered_df.withColumn("row_num", row_number().over(window))

# 2) 유저별 총 리뷰 개수 계산
user_counts = filtered_df.groupBy("userId").count().withColumnRenamed("count", "total_count")

# 3) 조인
df_with_counts = filtered_df.join(user_counts, on="userId")

# 4) train(60%), val(20%), test(20%) 분할 (row_num과 total_count를 이용해 정수 인덱스로 분할)
df_split = df_with_counts.withColumn(
    "dataset",
    when(col("row_num") <= floor(col("total_count") * 0.6), "train")
    .when(col("row_num") <= floor(col("total_count") * 0.8), "val")
    .otherwise("test")
)

# 5) 각 데이터셋 필터링 + 불필요 컬럼 삭제
train_df = df_split.filter(col("dataset") == "train").drop("row_num", "total_count", "dataset")
val_df = df_split.filter(col("dataset") == "val").drop("row_num", "total_count", "dataset")
test_df = df_split.filter(col("dataset") == "test").drop("row_num", "total_count", "dataset")

# 데이터 저장

In [0]:
train_df.write.mode("overwrite").saveAsTable("1dt_team8_databricks.final.train_df")
val_df.write.mode("overwrite").saveAsTable("1dt_team8_databricks.final.validation_df")
test_df.write.mode("overwrite").saveAsTable("1dt_team8_databricks.final.test_df")